<a href="https://colab.research.google.com/github/ChurikovSV/RAG_GigaChain/blob/main/%22RAG_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gigachain faiss-cpu sentence-transformers rank_bm25 playwright --quiet

In [ ]:
!pip install -q langchain
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
!playwright install

161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 12.7s161.3 MiB [] 0% 11.0s161.3 MiB [] 0% 8.1s161.3 MiB [] 0% 8.5s161.3 MiB [] 1% 8.3s161.3 MiB [] 1% 7.4s161.3 MiB [] 1% 7.2s161.3 MiB [] 1% 7.6s161.3 MiB [] 2% 7.9s161.3 MiB [] 2% 7.7s161.3 MiB [] 2% 7.4s161.3 MiB [] 2% 7.6s161.3 MiB [] 3% 8.0s161.3 MiB [] 3% 8.3s161.3 MiB [] 3% 7.6s161.3 MiB [] 4% 7.0s161.3 MiB [] 4% 6.6s161.3 MiB [] 5% 6.4s161.3 MiB [] 5% 6.3s161.3 MiB [] 5% 6.4s161.3 MiB [] 5% 6.5s161.3 MiB [] 6% 6.7s161.3 MiB [] 6% 6.9s161.3 MiB [] 6% 7.0s161.3 MiB [] 6% 7.1s161.3 MiB [] 6% 7.2s161.3 MiB [] 6% 7.4s161.3 MiB [] 6% 7.9s161.3 MiB [] 6% 8.1s161.3 MiB [] 7% 8.0s161.3 MiB [] 7% 7.9s161.3 MiB [] 8% 7.4s161.3 MiB [] 8% 7.0s161.3 MiB [] 9% 6.8s161.3 MiB [] 9% 6.6s161.3 MiB [] 9% 6.7s161.3 MiB [] 9% 6.8s161.3 MiB [] 10% 6.6s161.3 MiB [] 10% 6.5s161.3 MiB [] 10% 6.6s161.3 MiB [] 10% 6.7s161.3 MiB [] 11% 6.7s161.3 MiB [] 11% 6.6s161.3 MiB [] 12% 6.6s161.3 MiB [] 12% 6.5s161.3 MiB [] 13% 6.3s161.3 MiB [] 13% 6.1s161.3 MiB [] 14% 5.9s161.3 M

In [ ]:
%%writefile parser.py

from langchain_core.documents import Document
from langchain.document_loaders import AsyncChromiumLoader
from bs4 import BeautifulSoup

# Инициализируем загрузчик
loader = AsyncChromiumLoader(["https://nplus1.ru/search?tags=886"])
html = loader.load()
#articles = 'n1_climb_4 transition-colors duration-75 hover:text-main inline-block mb-10 sm:mb-5 font-spectral leading-24'
articles = 'n1_climb_4 transition-colors duration-75 hover:text-main inline-block mb-10 sm:mb-5 font-spectral leading-24'

with open('page.html', 'w') as f:
  f.write(html[0].page_content)
soup = BeautifulSoup(html[0].page_content, 'html.parser')
links = []
with open('links.txt', 'w') as f:
  for link in soup.find_all('a', class_=articles):
    print(link)
    links.append(link['href'])
    f.write(link['href']+' ')

Writing parser.py


In [ ]:
!python parser.py

/content/parser.py:3: LangChainDeprecationWarning: Importing AsyncChromiumLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import AsyncChromiumLoader

with new imports of:

>> from langchain_community.document_loaders import AsyncChromiumLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.document_loaders import AsyncChromiumLoader
USER_AGENT environment variable not set, consider setting it to identify your requests.
<a class="n1_climb_4 transition-colors duration-75 hover:text-main inline-block mb-10 sm:mb-5 font-spectral leading-24" href="https://nplus1.ru/news/2025/01/14/doom-in-pdf"><div>DOOM запустили на PDF</div><!-- --></a>
<a class="n1_climb_4 transition-colors duration-75 hover:text-main inline-block mb-10 sm:mb-5 font-spectral leading-24" href="https://nplus1.ru/news/2024/12/1

In [ ]:
with open('links.txt') as f:
  sources = f.read()

print(sources.split())

['https://nplus1.ru/news/2025/01/14/doom-in-pdf', 'https://nplus1.ru/news/2024/12/12/starlink-direct-to-cell-first-constellation-completed', 'https://nplus1.ru/news/2024/10/08/phys-nobel-2024', 'https://nplus1.ru/news/2024/08/15/fft-for-optical-neural-networks', 'https://nplus1.ru/blog/2024/06/13/stuck-on-the-platform', 'https://nplus1.ru/news/2024/05/08/alphafold-iii', 'https://nplus1.ru/news/2025/01/13/tetris', 'https://nplus1.ru/news/2024/12/04/gencast', 'https://nplus1.ru/news/2024/08/30/gamengen-google-deepmind', 'https://nplus1.ru/news/2024/05/15/gpt-4o', 'https://nplus1.ru/news/2024/04/25/neurocrispr', 'https://nplus1.ru/news/2025/01/13/ml-for-cancer-id', 'https://nplus1.ru/blog/2024/08/19/why-does-it-work', 'https://nplus1.ru/news/2024/07/26/alphaproof-alphageometry2-deepmind', 'https://nplus1.ru/news/2024/05/14/apple-google-unwanted-tracking-alerts', 'https://nplus1.ru/news/2024/11/08/neiry-pifia', 'https://nplus1.ru/news/2024/08/15/ioai-2024', 'https://nplus1.ru/news/2024/06/

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [ ]:
loader = WebBaseLoader(sources.split())
docs = loader.load()
len(docs)

20

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)
len(split_docs)

409

In [ ]:
split_docs[1]

Document(metadata={'source': 'https://nplus1.ru/news/2025/01/14/doom-in-pdf', 'title': 'DOOM запустили на\xa0PDF. Автора вдохновил тетрис внутри PDF', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='Космонавтика\nАрхеология\nНейронауки\nНа мышах\nЗвук\nКрасота\n\n\nНаучные закрытия\nИИ спешит на помощь\nКогда рассеется дым\nКнижная полка\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                О нас\n              \n\n                Сложность\n              \n\n                Рекламодателям\n              \n\n                Авторам')

In [ ]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(split_docs, embedding=embedding)

<timed exec>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 2min 35s, sys: 12.5 s, total: 2min 48s
Wall time: 3min 11s


In [ ]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Создание генеративной части пайплайна и составление цепочки RAG системы

In [ ]:
# импортируем ключ, который получили для гигачата
from google.colab import userdata
auth = userdata.get('AUTH_KEY')

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [ ]:
llm = GigaChat(credentials=auth,
              model='GigaChat:latest',
               verify_ssl_certs=False,
               profanity_check=False)
prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

<ipython-input-18-7895cbad50b3>:1: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(credentials=auth,


In [ ]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
    )

In [ ]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

In [ ]:
q1 = 'Как машинное обучение упростило диагностику рака?'
resp1 = retrieval_chain.invoke(
    {'input': q1}
)


In [ ]:
resp1

{'input': 'Как машинное обучение упростило диагностику рака?',
 'context': [Document(id='44f70ea3-8132-4205-9d54-2d2891e25c73', metadata={'source': 'https://nplus1.ru/news/2025/01/13/ml-for-cancer-id', 'title': 'Машинное обучение упростило диагностику рака по\xa0подтипам. Модели-классификаторы выложили в\xa0открытый доступ', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='Машинное обучение упростило диагностику рака по\xa0подтипам. Модели-классификаторы выложили в\xa0открытый доступ\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nN + offline\nN + production\n\n\n\n\n                О нас\n              \n\n                Сложность\n              \n\n                Рекламодателям\n              \n\n                Авторам\n              \n\n\nИгры\nФизика\nЗоология\nАстрономия\nГенетика\nМатематика'),
  Document(id

In [ ]:
q2 = 'Когда Хаббл вернулся к работе?'
resp2 = retrieval_chain.invoke(
    {'input': q2}
)
resp2

{'input': 'Когда Хаббл вернулся к работе?',
 'context': [Document(id='35c21ec4-932e-4e91-a855-02820200ac57', metadata={'source': 'https://nplus1.ru/news/2024/12/10/spacex-super-heavy-static-fire-test-for-ift7', 'title': 'Первая ступень Starship успешно прошла прожиг перед седьмым испытательным полетом. Запуск может состояться уже через месяц', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='и\xa0посадки на\xa0башню обслуживания была направлена по\xa0альтернативной траектории в\xa0Мексиканский залив. Как стало известно позднее, возвращению ускорителя помешала неисправность оборудования башни обслуживания, которую выявила автоматическая процедура проверки во\xa0время фазы торможения после разделения ступеней.'),
  Document(id='8ec7f0cd-99ce-4b80-aa6c-2e7caa002791', metadata={'source': 'https://nplus1.ru/news/2024/12/10/spacex-super-heavy-static-fire-test-for-ift7', 'title': 'Первая ступень Starship успешно прошла прожиг перед седь

In [ ]:
q3 = 'Какая есть информация о неудачных запусках?'
resp3 = retrieval_chain.invoke(
    {'input': q3}
)
resp3

{'input': 'Какая есть информация о неудачных запусках?',
 'context': [Document(id='cee62342-5956-4b71-8a0f-2853bfb1c724', metadata={'source': 'https://nplus1.ru/blog/2024/12/11/the-mythology-of-the-soviet-space', 'title': '«Мифология советского космоса». Почему космонавтов обучали действовать подобно машинам', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='Деятельность космонавта на\xa0орбите планировалась детально и\xa0тщательно. Время и\xa0длительность каждого действия определялись на\xa0Земле заранее. За\xa0каждое отклонение от\xa0установленной процедуры во\xa0время полета космонавты получали замечание. Ошибкой считалось малейшее отклонение, например неверное переключение тумблера, даже если оно не\xa0влияло на\xa0работу системы. В\xa0среднем экипаж из\xa0двух человек получал\xa0— замечаний за\xa0время орбитального полета, длившегося несколько месяцев. Это значит, что каждый член'),
  Document(id='35c21ec4-932e-4e91-a855-028

In [ ]:
q4 = 'Есть ли какая-то информация, связанная с именем древнегреческого математика?'
resp4 = retrieval_chain.invoke(
    {'input': q4}
)
resp4

{'input': 'Есть ли какая-то информация, связанная с именем древнегреческого математика?',
 'context': [Document(id='47ed793e-e5a0-4c69-afab-ae91c56ae8aa', metadata={'source': 'https://nplus1.ru/news/2024/12/02/tmds-system', 'title': 'Роботизированная платформа с\xa0лебедкой поможет робоспелеологу спуститься в\xa0инопланетную пещеру. Она позволяет роботу отсоединяться после спуска для продолжения исследований в\xa0глубине', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='Пещеры вулканического происхождения на\xa0Марсе и\xa0Луне привлекают пристальное внимание ученых. Благодаря защищенности от\xa0метеоритных бомбардировок, экстремальных колебаний температуры, ультрафиолетового излучения и\xa0космических заряженных частиц, эти полости могли сохранить в\xa0себе важную информацию о\xa0геологической истории небесного тела и\xa0возможные следы внеземной жизни. Кроме того, инопланетные пещеры рассматриваются как потенциальные места, где

In [ ]:
q5 = "Дай информацию, связанную с полями."
resp5 = retrieval_chain.invoke(
    {'input': q5}
)
resp5

{'input': 'Дай информацию, связанную с полями.',
 'context': [Document(id='47ed793e-e5a0-4c69-afab-ae91c56ae8aa', metadata={'source': 'https://nplus1.ru/news/2024/12/02/tmds-system', 'title': 'Роботизированная платформа с\xa0лебедкой поможет робоспелеологу спуститься в\xa0инопланетную пещеру. Она позволяет роботу отсоединяться после спуска для продолжения исследований в\xa0глубине', 'description': 'N + 1 — главное издание о науке, технике и технологиях', 'language': 'ru'}, page_content='Пещеры вулканического происхождения на\xa0Марсе и\xa0Луне привлекают пристальное внимание ученых. Благодаря защищенности от\xa0метеоритных бомбардировок, экстремальных колебаний температуры, ультрафиолетового излучения и\xa0космических заряженных частиц, эти полости могли сохранить в\xa0себе важную информацию о\xa0геологической истории небесного тела и\xa0возможные следы внеземной жизни. Кроме того, инопланетные пещеры рассматриваются как потенциальные места, где в\xa0будущем могли\xa0бы разместиться ба